In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text2text-generation", model="eugenesiow/bart-paraphrase")

In [ ]:
# # Your prompt
# prompt = ""
# # Generate a response using the pipeline
# response = pipe(prompt, max_new_tokens=8)

# # The response is a list of dictionaries, each containing the generated text.
# # We'll print the first response's generated text.
# print(response[0]['generated_text'])

In [37]:
# Importing necessary libraries
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import pandas as pd  
import mediapipe as mp

In [38]:
dir_path = os.path.join(os.getcwd(), 'dataset')
wlas_path = os.path.join(dir_path, 'WLASL_v0.3.json')
wlas_df = pd.read_json(wlas_path)

In [39]:
wlas_df

,gloss,instances
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra..."
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
...,...,...
1995,washington,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
1996,waterfall,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
1997,weigh,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
1998,wheelchair,"[{'bbox': [415, 86, 1811, 1080], 'fps': 25, 'f..."


In [51]:
import os
import json
import pandas as pd

# Assuming main_path is defined in your notebook, pointing to the dataset directory
dir_path = os.path.join(os.getcwd(), 'dataset')

# Function to get video IDs if the video file exists
def fetch_video_ids(instances):
    video_ids = []
    for instance in instances:
        video_id = instance['video_id']
        if os.path.exists(f'{dir_path}/videos/{video_id}.mp4'):
            video_ids.append(video_id)
    return video_ids

In [ ]:

# # Function to get JSON features including video IDs, URLs, and file paths for existing videos
# def get_json_features(instances):
#     video_ids = []
#     video_urls = []
#     file_paths = []
#     for instance in instances:
#         video_id = instance['video_id']
#         video_url = instance.get('url', '')  # Assuming 'url' key exists and defaulting to '' if not
#         file_path = f'{main_path}videos/{video_id}.mp4'
#         if os.path.exists(file_path):
#             video_ids.append(video_id)
#             video_urls.append(video_url)
#             file_paths.append(file_path)
#     return video_ids, video_urls, file_paths



# Example of how to use get_json_features (if needed)
# This is not directly applied to the DataFrame in the provided explanation but could be used similarly
# for video_id, video_url, file_path in zip(*get_json_features(wlas_df['instances'])):
#     print(video_id, video_url, file_path)

In [52]:
# Applying get_videos_ids function to add 'videos_ids' column to the DataFrame
wlas_df['video_ids'] = wlas_df['instances'].apply(fetch_video_ids)

In [54]:
df = wlas_df.explode('video_ids')

In [56]:
df = df[['gloss', 'video_ids']]

In [57]:
df

,gloss,video_ids
0,book,69241
0,book,07069
0,book,07068
0,book,07070
0,book,07099
...,...,...
1998,wheelchair,63047
1998,wheelchair,63050
1999,whistle,63186
1999,whistle,63188


In [58]:
df.to_csv('wlasl_video_ids.csv', index=False)

In [61]:
# counting the number of samples for each gloss
df['samples_num'] = df['video_ids'].apply(len)

In [62]:
# Define a custom function to generate the path, participant_id, sign, and sequence_id columns
def process_row(row):
    video_id = row["video_ids"]
    path = f'{dir_path}/videos/{video_id}.mp4'  
    participant_id = video_id
    sign = row["gloss"]
    sequence_id = row["samples_num"]
    return pd.Series([path, participant_id, sign, sequence_id])

In [63]:
# Applying the custom function to the DataFrame
train_df = df.apply(process_row, axis=1)

In [65]:
# using direct assignment to name the columns
train_df.columns = ["path", "participant_id", "sign", "sequence_id"]

In [68]:
train_df

,path,participant_id,sign,sequence_id
0,d:\NullClass_Internship\Sign_Language_Predicti...,69241,book,5
0,d:\NullClass_Internship\Sign_Language_Predicti...,07069,book,5
0,d:\NullClass_Internship\Sign_Language_Predicti...,07068,book,5
0,d:\NullClass_Internship\Sign_Language_Predicti...,07070,book,5
0,d:\NullClass_Internship\Sign_Language_Predicti...,07099,book,5
...,...,...,...,...
1998,d:\NullClass_Internship\Sign_Language_Predicti...,63047,wheelchair,5
1998,d:\NullClass_Internship\Sign_Language_Predicti...,63050,wheelchair,5
1999,d:\NullClass_Internship\Sign_Language_Predicti...,63186,whistle,5
1999,d:\NullClass_Internship\Sign_Language_Predicti...,63188,whistle,5
